In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score
import numpy as np

# Veri setini yükleme
train_data_dir = r"C:\Users\golde\Desktop\data2\Data\train"
validation_data_dir = r"C:\Users\golde\Desktop\data2\Data\valid"
test_data_dir = r"C:\Users\golde\Desktop\data2\Data\test"

# Veri ön işleme işlemleri için ImageDataGenerator kullanımı
train_datagen = ImageDataGenerator(rescale=1./255)  
validation_datagen = ImageDataGenerator(rescale=1./255)  
test_datagen = ImageDataGenerator(rescale=1./255)  

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical')

# DenseNet modeli
def dense_block(x, blocks, growth_rate):
    for _ in range(blocks):
        y = BatchNormalization()(x)
        y = Activation('relu')(y)
        y = Conv2D(4 * growth_rate, 3, padding='same')(y)
        x = Concatenate()([x, y])
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(growth_rate, 1, padding='same')(x)
    return x

def transition_block(x, reduction):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(int(reduction * x.shape[-1]), 1, padding='same')(x)
    x = tf.keras.layers.AveragePooling2D(2, strides=2)(x)
    return x

def densenet(input_shape, num_classes, blocks_per_group, growth_rate, reduction):
    inputs = Input(input_shape)

    x = Conv2D(2 * growth_rate, 7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D(3, strides=2, padding='same')(x)

    num_blocks = len(blocks_per_group)
    for i, blocks in enumerate(blocks_per_group):
        x = dense_block(x, blocks, growth_rate)
        if i != num_blocks - 1:
            x = transition_block(x, reduction)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)

    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

input_shape = (150, 150, 3)
num_classes = 2
blocks_per_group = [6, 12, 24, 16]
growth_rate = 32
reduction = 0.5

model = densenet(input_shape, num_classes, blocks_per_group, growth_rate, reduction)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=30,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))

test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('Test accuracy:', test_acc)

# Tahminlerin alınması
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

# Performans metriklerinin hesaplanması
cm = confusion_matrix(y_true, y_pred)
classification_report = classification_report(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(classification_report)
print("ROC AUC Score:", roc_auc)
print("F1 Score:", f1)


model.save('densenet_model.h5')
